## Converting from OneMap Singapore's [X Y coordinates](https://www.onemap.gov.sg/docs/#3414-svy21-to-3857) in 3414(SVY21) format to latitude and longitude in 4326(WGS84) format  

Information loads very quickly when the url is opened but the information took very long to be retrieved using the requests.get function. Hence instead of using the OneMap API to convert the coordinates, here's a much quicker alternative. 

In [1]:
import webbrowser

In [ ]:
url = "https://developers.onemap.sg/commonapi/convert/3414to4326?X=28983.788791079794&Y=33554.5098132845"

In [7]:
webbrowser.open(url)

True

## Using requests.get

In [10]:
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup

In [12]:
import time

In [4]:
hdr = {'User-Agent': 'Mozilla/5.0'}

In [3]:
x_list = [28983.78879107979]
y_list = [33554.5098132845]

In [7]:
for i in range(0,1):
    url = "https://developers.onemap.sg/commonapi/convert/3414to4326?X={}4&Y={}".format(x_list[i],y_list[i])
    start = time.time()
    print('retrieving...')
    bookpage = requests.get(url, headers=hdr)
    end = time.time()
    soup = BeautifulSoup(bookpage.text, "html.parser")
    print(end - start)

retrieving...
229.76258301734924


In [19]:
soup

{"latitude":1.3197295716669164,"longitude":103.84215843333567}

In [29]:
list_latlong = []

In [9]:
list_latlong.append((x,y,soup))

In [10]:
list_latlong

[(28983.788791079794,
  33554.5098132845,
  {"latitude":1.3197295716669164,"longitude":103.84215843333567})]

## Using pyproj - better alternative

In [11]:
import pyproj

In [16]:
from pyproj import Transformer

In [21]:
import pyproj
p = pyproj.CRS("epsg:3414")
p_to = pyproj.CRS("epsg:4326")  

### Applying to a list

In [28]:
def xy_to_lonlat(x, y):
    transformer = pyproj.Transformer.from_crs(p, p_to, always_xy=True)
    lonlat = transformer.transform(x, y)
    return lonlat[0], lonlat[1]

In [ ]:
x_list = [28983.78879107979]
y_list = [33554.5098132845]

In [31]:
for i in range(0,1):
    lon, lat = xy_to_lonlat(x_list[i],y_list[i])

In [32]:
print(lat,lon)

1.3197295716669195 103.84215843333536


### Applying to a pandas dataframe

In [0]:
def xy_to_lonlat(x, y):
    transformer = pyproj.Transformer.from_crs(p, p_to, always_xy=True)
    lonlat = transformer.transform(x, y)
    return lonlat#lonlat[0], lonlat[1]

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
df["lonlat"] = df.apply(lambda x: xy_to_lonlat(x.x_coord, x.y_coord), axis=1)

In [0]:
df['lonlat']= df['lonlat'].astype(str)

In [ ]:
df[['longitude','latitude']] = df['lonlat'].str.split(',',expand=True)

In [ ]:
df['longitude'] = df['longitude'].str.replace('(','')
df['latitude'] = df['latitude'].str.replace(')','')

References:  
https://github.com/pyproj4/pyproj/issues/537  